In [5]:
'''
Sample Command:-
python generate_aruco_tags.py --id 24 --type DICT_5X5_100 -o tags/
'''
import numpy as np
import argparse
import cv2
import sys
import time
from scipy.spatial.transform import Rotation as R

from IPython.display import clear_output



# for 1920, 1080
intrinsic_mat = np.array([
 [1.16249999e+03, 0.00000000e+00, 9.72336017e+02],
 [0.00000000e+00, 1.15780995e+03, 5.65214826e+02],
 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

dist_coeff = np.array([[0.085, -0.199, -0.00065, 0.00063, 0.0469]])


In [6]:
# pip uninstall opencv-python-headless==4.2.0.34

# pip3 uninstall opencv-python

#pip3 uninstall opencv-contrib-python
#pip3 install opencv-contrib-python


In [9]:
ARUCO_DICT = {
"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

tag_size = 350
id_ = 0
type_ = cv2.aruco.DICT_5X5_50

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
# parameters =  cv.aruco.DetectorParameters()
# detector = cv.aruco.ArucoDetector(dictionary, parameters)


# arucoDict = cv2.aruco.Dictionary_get(type_)

tag = np.zeros((tag_size, tag_size, 1), dtype="uint8")
cv2.aruco.generateImageMarker(dictionary, id_, tag_size, tag, 1)

# Save the tag generated
tag_name = f'{type_}_id_{id_}_{tag_size}.png'
print(tag_name)
cv2.imwrite(tag_name, tag)
cv2.imshow("ArUCo Tag", tag)
cv2.waitKey(0)
cv2.destroyAllWindows()

4_id_0_350.png


In [10]:
def draw_corners(aruco_dict_type):
    
    parameters = cv2.aruco.DetectorParameters()
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
#     aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    cap = cv2.VideoCapture(0)#, cv2.CAP_DSHOW)
    # cap.set(3, 1920)
    # cap.set(4, 1080)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video stream.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break
        
        # Convert to grayscale
        cv2.imshow('ArUco Marker Detection', frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        

        # Detect ArUco markers
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
        

        # If markers are detected, draw them
        if ids is not None:
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)
#             print(corners)

        # Display the resulting frame
        cv2.imshow('ArUco Marker Detection', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

    
draw_corners(cv2.aruco.DICT_5X5_50)


In [2]:
def pose_esitmation(aruco_dict_type, matrix_coefficients, distortion_coefficients):
    # Open the video stream (replace with file path if using a video file)
    video = cv2.VideoCapture(0)#, cv2.CAP_DSHOW)  # 0 for webcam, or replace with video file path

    # Define the ArUco dictionary to use
#     aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)  # Choose an appropriate dictionary
    parameters = cv2.aruco.DetectorParameters()

    
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
#     parameters = cv2.aruco.DetectorParameters_create()
    
    while True:
        # Capture frame-by-frame
        clear_output(wait=True)
        ret, frame = video.read()
        if not ret:
            print("No video")
            break

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect ArUco markers
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        # If markers are detected, draw them
        if ids is not None:
            for i in range(0, len(ids)):
                cv2.aruco.drawDetectedMarkers(frame, corners, ids)
                rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.06, matrix_coefficients, distortion_coefficients)
                # Draw a square around the markers
                cv2.aruco.drawDetectedMarkers(frame, corners) 

                # Draw Axis
                cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.06)
                RR= R.from_rotvec(rvec[0])
                rvec = np.array(RR.as_matrix())
                print("rvec: ", rvec)
                print("tvec: ", tvec)
        # Display the resulting frame
        cv2.imshow('ArUco Marker Detection', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close windows
    video.release()
    cv2.destroyAllWindows()

    
pose_esitmation(cv2.aruco.DICT_5X5_50,intrinsic_mat, dist_coeff)


rvec:  [[[ 0.85821187  0.44875201  0.24918674]
  [ 0.50958713 -0.68662571 -0.51852299]
  [-0.06159021  0.57198494 -0.81794858]]]
tvec:  [[[-0.36675454 -0.21423287  0.83205791]]]
rvec:  [[[ 0.77869523  0.54923854  0.30326683]
  [ 0.60388667 -0.52503598 -0.59971502]
  [-0.1701606   0.65013402 -0.74052085]]]
tvec:  [[[-0.41394046 -0.14342836  0.7474721 ]]]


In [ ]:
%pip install ultralytics

In [4]:
import cv2
from ultralytics import YOLO
import numpy as np

model = YOLO('yolov8x.pt')
cap = cv2.VideoCapture(0)#, cv2.CAP_DSHOW)
# cap.set(3, 1920)
# cap.set(4, 1080)
if not cap.isOpened():
    raise ValueError("Error: Could not open video stream.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    results = model(frame)
    current_detections = []

    # Extract detections from YOLO results
    for result in results:
        for box in result.boxes:
            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            # Calculate center of the bounding box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            
            
            # Extract label and confidence score
            label = model.names[int(box.cls[0])]
            confidence = box.conf[0].item()
            print(label)
            print("x:", center_x, "y:",center_y)


            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)  # Draw center point

        # Show the frame with bounding boxes
    cv2.imshow("YOLO Live Detection", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

/home/lab/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0



0: 480x640 1 person, 2 cups, 1 tv, 1 laptop, 1480.4ms
Speed: 7.8ms preprocess, 1480.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
person
x: 138 y: 140
cup
x: 354 y: 108
cup
x: 544 y: 116
tv
x: 282 y: 56
laptop
x: 319 y: 343

0: 480x640 1 person, 2 cups, 1 tv, 1 laptop, 1154.7ms
Speed: 1.5ms preprocess, 1154.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
person
x: 138 y: 140
cup
x: 354 y: 99
cup
x: 544 y: 116
tv
x: 282 y: 56
laptop
x: 319 y: 344

0: 480x640 1 person, 2 cups, 1 tv, 1 laptop, 1 vase, 1249.5ms
Speed: 29.7ms preprocess, 1249.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
person
x: 138 y: 140
cup
x: 354 y: 100
cup
x: 544 y: 116
vase
x: 544 y: 116
tv
x: 280 y: 56
laptop
x: 319 y: 343

0: 480x640 1 person, 2 cups, 1 tv, 1 laptop, 1 book, 1 vase, 1305.6ms
Speed: 28.3ms preprocess, 1305.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
person
x: 138 y: 140
cup
x: 354 y: 109
cup
x: 544 y: 116
v

In [ ]:
%pip install python3-opencv